# Intermediate Event Models

In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

In [ ]:
PROJ_ROOT = Path().resolve().parents[3]
env_file_dir = PROJ_ROOT / '.env'
_ = load_dotenv(env_file_dir, verbose=True)

## About

Develop an **intermediate** model that can be used to calculate e-commerce product metrics from the user events on the Greenery website in downstream fact product models.

Some metrics per product that should be supported are

1. daily
   - <span style="color: darkred">page views</span> (<span style="color: red">required</span>)
   - <span style="color: green">orders</span> (<span style="color: red">required</span>)
   - <span style="color: purple">sessions</span> (<span style="color: darkgreen">optional</span>)
2. overall
   - <span style="color: dodgerblue">high traffic but low purchases (conversions)</span> (<span style="color: red">required</span>)
   - <span style="color: teal">cart abandonment rate</span> (<span style="color: darkgreen">optional</span>)
   - <span style="color: magenta">average time on page</span> (<span style="color: darkgreen">optional</span>)

The models to be developed, and their columns, are

1. `products/int_products_daily_totals`
   - number of daily orders per product
   - number of daily page views per product
   - number of daily sessions per product, in which that product was viewed
   - used to calculate
     - <span style="color: darkred">daily page views per product</span>
     - <span style="color: green">daily orders per product</span>
     - <span style="color: purple">daily sessions per product</span>
2. `products/int_events_aggregated_to_product`
   - number of sessions not ending in a purchase, in which a product page was viewed, per product
   - number of sessions ending in a purchase, in which a product page was viewed, per product
   - number of purchases, per product
   - used to calculate
     - <span style="color: dodgerblue">high traffic but low purchases (conversions)</span>
3. `products/int_products_purchase_abandoned_cart_sessions_summed`
   - number of times product was added to cart, per session
   - number of times product was purchased, per session
   - used to calculate
     - <span style="color: teal">cart abandonment rate per product</span>
4. `products/int_products_page_viewing_time_averaged`
   - average time spent viewing product page across all sessions, per product
   - standard deviation of time spent viewing product page across all sessions, per product
   - used to calculate
     - <span style="color: magenta">average time on page</span>

### Use-Case

This `product_id` level aggregation will be performed here so it does not need to repeated by the following marts models

1. `marts/marketing/products/fct_products`
2. `marts/marketing/products/daily/fct_products_daily`

### Notes

1. This notebook supports <kbd>Run</kbd> > <kbd>Run All Cells</kbd>.

## User Inputs

In [ ]:
#

In [ ]:
engine = create_engine(
    URL(
        drivername="driver",
        account=os.getenv("UPLIMIT_SNOWFLAKE_ACCOUNT"),
        user=os.getenv("UPLIMIT_SNOWFLAKE_USER"),
        password=os.getenv("UPLIMIT_SNOWFLAKE_PASS"),
        warehouse=os.getenv("UPLIMIT_SNOWFLAKE_WAREHOUSE"),
        role=os.getenv("UPLIMIT_SNOWFLAKE_ROLE"),
        database=os.getenv("UPLIMIT_SNOWFLAKE_DB_NAME"),
        schema=os.getenv("UPLIMIT_SNOWFLAKE_SCHEMA"),
    )
)

## Connect

Load Jupyter SQL extension

In [ ]:
%load_ext sql

Connect to DuckDB database

In [ ]:
%sql engine --alias connection

## Exploratory Data Analysis

Show the different types of events

In [ ]:
%%sql
SELECT DISTINCT(event_type) AS event_type
FROM stg_postgres_events

Show the first few rows of the events table

In [ ]:
%%sql
SELECT *
FROM stg_postgres_events
LIMIT 4

Set the maximum number of rows to be displayed to `None` (shows all rows)

In [ ]:
%config SqlMagic.displaylimit = None

Show chronolgically ordered events in 15 sessions in which there was no checkout or package shipment

In [ ]:
%%sql
WITH t1 AS (
    SELECT * EXCLUDE(event_id),
           ROW_NUMBER() OVER(
               PARTITION BY session_id ORDER BY user_id, created_at DESC
           ) AS rn
    FROM stg_postgres_events
),
t2 AS (
    SELECT session_id
    FROM t1
    -- get sessions in which last event (rn = 1) is not shipping or checkout
    WHERE (rn = 1 AND event_type NOT IN ('checkout', 'package_shipped'))
    ORDER BY session_id
    LIMIT 15
),
sessions_without_purchase AS (
    SELECT * EXCLUDE(event_id)
    FROM stg_postgres_events
    -- user INNER JOIN to only get sessions in which order was not placed
    INNER JOIN t2 USING (session_id)
    ORDER BY session_id, user_id, created_at
)
SELECT *
FROM sessions_without_purchase

A single session is associated with a single user. There are no multi-user sessions

In [ ]:
%%sql
WITH multi_user_sessions AS (
    SELECT session_id,
           COUNT(DISTINCT(user_id)) AS num_users
    FROM stg_postgres_events
    GROUP BY ALL
    HAVING num_users > 1
)
SELECT COUNT(*) AS num_multi_user_sessions
FROM multi_user_sessions

A single user can have multiple sessions

In [ ]:
%%sql
WITH sessions_per_user AS (
    SELECT user_id,
           COUNT(DISTINCT(session_id)) AS num_sessions
    FROM stg_postgres_events
    GROUP BY ALL
),
single_multi_session_users AS (
    SELECT *
    FROM (
        SELECT '1' AS num_sessions,
               COUNT(*) AS num_users,
               False AS is_multi_session_user
        FROM sessions_per_user
        WHERE num_sessions = 1
        GROUP BY ALL
    )
    UNION ALL
    SELECT '1+' AS num_sessions,
           COUNT(*) AS num_users,
           True AS is_multi_session_user
    FROM sessions_per_user
    WHERE num_sessions > 1
)
SELECT *
FROM single_multi_session_users

As seen from the total of the `num_users` column in the above query result, all users in the `users` table do not have an entry in the `events` table. This is verified below

In [ ]:
%%sql
WITH num_users_from_events AS (
    SELECT COUNT(DISTINCT(user_id)) AS num_users_events,
           1 AS row_num
    FROM stg_postgres_events
),
num_users_overall AS (
    SELECT COUNT(DISTINCT(user_id)) AS num_users,
           1 AS row_num
    FROM stg_postgres_users
)
SELECT *
FROM num_users_from_events
INNER JOIN num_users_overall USING (row_num)

Get bounce events (Method 1)

In [ ]:
%%sql
/* get sessions with zero non page view events */
SELECT user_id,
       session_id,
       event_type,
       COUNT(*) AS num_events
FROM stg_postgres_events
GROUP BY ALL
HAVING (event_type = 'add_to_cart' AND num_events = 0)
AND (event_type = 'checkout' AND num_events = 0)
AND (event_type = 'package_shipped' AND num_events = 0)

Get bounce events (Method 2)

In [ ]:
%%sql
/* get single page view sessions */
WITH single_page_view_sessions AS (
    SELECT user_id,
           session_id,
           event_type,
           COUNT(*) AS num_events
    FROM stg_postgres_events
    GROUP BY ALL
    HAVING (event_type = 'page_view' AND num_events = 1)
)
/* get sessions with zero non page view events */
SELECT e.user_id,
       e.session_id,
       COUNT(*) AS num_non_page_view_events
FROM stg_postgres_events e
INNER JOIN single_page_view_sessions ss USING (session_id)
WHERE event_type != 'page_view'
GROUP BY ALL
HAVING num_non_page_view_events = 0

There are no [bounce events](https://support.google.com/analytics/answer/1009409?hl=en) as shown using two methods above, so it is not necessary to calculate the bounce rate metric per product or at any other level.

## Models

### `products/int_products_daily_totals`

Show chronolgically ordered events in four sessions in which there was a checkout

In [ ]:
%%sql
SELECT * EXCLUDE(event_id)
FROM stg_postgres_events
WHERE session_id IN (
    '00b3765f-3af3-4cbf-9fba-904daf17faff',
    'a646b9aa-0044-4fbd-8bae-a1018f5d4ace',
    'dc881be8-fee9-4f48-bdf0-fd0435a8d4af',
    'fc449603-99a9-4a74-a5b7-14659e06722b'
)
ORDER BY session_id, user_id, created_at

**Observations**

1. The last event in sessions that end in a purchase is one of
   - `package_shipped`
   - `checkout`
   but the `product_id` is missing in both of these types of events.
2. In every session with a purchase
   - every product that is purchased must *first* be added to the cart
   - the `event_type = 'add_to_cart'` filter indicates which products have been purchased

In [ ]:
%%sql
/* get add-to-cart events from sessions in which a product was purchased */
WITH products_ordered AS (
    SELECT product_id,
           created_at
    FROM int_product_purchases_filtered
    -- get add-to-cart events since only products in a cart can be purchased
    WHERE event_type = 'add_to_cart'
),
/* count number of daily orders (purchases) per product */
daily_orders_by_product AS (
    SELECT product_id,
           TO_DATE(created_at) AS created_at_date,
           COUNT(*) AS num_orders
    FROM products_ordered
    GROUP BY ALL
),
/* get page view events from all sessions */
daily_page_views_per_product_session AS (
    SELECT session_id,
           product_id,
           TO_DATE(created_at) AS created_at_date
    FROM stg_postgres_events
    WHERE event_type = 'page_view'
),
/* count number of daily page views and daily sessions per product */
daily_page_views_by AS (
    SELECT product_id,
           created_at_date,
           COUNT(*) AS num_page_views,
           COUNT(DISTINCT(session_id)) AS num_sessions
    FROM daily_page_views_per_product_session
    GROUP BY ALL
),
/* combine daily sums (orders, page views, sessions) per product */
daily_product_totals AS (
    SELECT pv.product_id,
           pv.created_at_date,
           IFNULL(pv.num_page_views, 0) AS num_page_views,
           IFNULL(pv.num_sessions, 0) AS num_sessions,
           IFNULL(dor.num_orders, 0) AS num_orders
    FROM daily_page_views_by pv
    LEFT JOIN daily_orders_by_product dor USING (product_id, created_at_date)
)
SELECT *
FROM daily_product_totals

**Notes**

1. In the `product_ordered` CTE, the `WHERE event_type = 'add_to_cart'` clause gets events corresponding to products added to cart. As shown earlier, products must first be added to a cart before they can be purchased in a single session. Since the result of this CTE will be used in the `daily_orders_by_product` CTE to count purchases per product, we only need these add-to-cart events since only products that were first added to a cart can be purchased. So, these events capture the purchase of a product during a session. For this reason, the `WHERE event_type = 'add_to_cart'` clause is needed in this CTE.

### `products/int_events_sessions_aggregated_to_product`

[Web traffic is measured in sessions](https://victorious.com/blog/measure-site-traffic/)

> You can measure website traffic ... with the following three metrics:

       > 1. Users

       > 2. Sessions (this is used here)

       >  ...

In [ ]:
%%sql
/* get events for sessions that did not end in a purchase */
WITH products_non_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_non_purchases_filtered
),
/* get events for sessions that did end in (convert to) a purchase */
products_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_purchases_filtered
),
/* count number of sessions not ending in a purchase in which product page was
viewed */
product_non_purchase_page_views AS (
    SELECT product_id,
           COUNT(*) AS num_non_purchase_page_views,
           COUNT(DISTINCT(session_id)) AS num_non_purchase_page_view_sessions
    FROM products_non_purchase_sessions
    -- get the page view event from sessions in which product was not ordered
    WHERE event_type = 'page_view'
    GROUP BY product_id
),
/* count number of sessions ending in a purchase in which product page was
viewed */
product_purchase_page_views AS (
    SELECT product_id,
           COUNT(*) AS num_purchase_page_views,
           COUNT(DISTINCT(session_id)) AS num_purchase_page_view_sessions
    FROM products_purchase_sessions
    -- get the page view event from sessions in which product was ordered
    WHERE event_type = 'page_view'
    GROUP BY product_id
),
/* count purchases */
product_purchases AS (
    SELECT product_id,
           COUNT(DISTINCT(session_id)) AS num_purchases
    FROM products_purchase_sessions
    -- get add-to-cart events since only products in a cart can be purchased
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* join the three types of session counts per product */
product_session_totals AS (
    SELECT pp.product_id,
           -- count of sessions with page view but not ending in product
           -- purchase
           npv.num_non_purchase_page_view_sessions,
           -- count of sessions with page view and ending in product purchase
           ppv.num_purchase_page_view_sessions,
           -- count of product purchases
           pp.num_purchases,
           -- count of total page views
           (
               npv.num_non_purchase_page_views
               + ppv.num_purchase_page_views
           ) AS num_page_views
    FROM product_non_purchase_page_views npv
    INNER JOIN product_purchases pp USING (product_id)
    INNER JOIN product_purchase_page_views ppv USING (product_id)
)
SELECT *
FROM product_session_totals

**Notes**

1. In the `product_purchases` CTE, the `WHERE event_type = 'add_to_cart'` clause gets events corresponding to products added to cart. Since this CTE is looking to count purchases per product, we only need these add-to-cart events since only products that were first added to a cart can be purchased. So, these events capture the purchase of a product during a session. Products that were viewed but not added to a cart before a purchase would not be included in the purchase and are excluded. For this reaso, the the `WHERE event_type = 'add_to_cart'` clause is needed in this CTE.

#### Sanity Check

Verify the two types of page view totals for two sessions

In [ ]:
%%sql
WITH t1 AS (
    SELECT session_id,
           product_id,
           event_type
    FROM stg_postgres_events
    WHERE product_id IN (
        '64d39754-03e4-4fa0-b1ea-5f4293315f67',
        'e706ab70-b396-4d30-a6b2-a1ccf3625b52'
    )
    AND event_type = 'page_view'
)
SELECT product_id,
       COUNT(DISTINCT(session_id)) AS num_page_view_sessions,
       COUNT(*) AS num_page_views
FROM t1
GROUP BY ALL

**Observations**

1. Some product pages were viewed once per session, so the number of page views and number of page view sessions are equal.
2. Some product pages were viewed multiple times per session, so the number of page view sessions is less than the number of page views.

### `products/int_products_purchase_abandoned_cart_sessions_summed`

Calculate the Cart Abandonment Rate ([1](https://www.geckoboard.com/best-practice/kpi-examples/shopping-cart-abandonment-rate/), [2](https://www.bigcommerce.com/articles/ecommerce/abandoned-carts/)) per product

1. Count number of add-to-carts performed per product in each session
2. Count number of add-to-carts performed per product in each session in which a purchase occurred

#### Include Number of Purchases

The number of purchases per product is included here even though it is captured in other intermediate queries

In [ ]:
%%sql
/* get events for sessions that did not end in a purchase */
WITH products_non_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_non_purchases_filtered
),
/* get events for sessions that did end in (convert to) a purchase */
products_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_purchases_filtered
),
/* count number of sessions not ending in a purchase in which product page was
viewed */
product_non_purchase_add_to_cart AS (
    SELECT product_id,
           COUNT(*) AS num_non_purchase_add_to_carts
    FROM products_non_purchase_sessions
    -- get the add-to-cart event from sessions in which product was added to
    -- a cart
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* count number of sessions ending in a purchase in which product page was
viewed */
product_purchase_add_to_cart AS (
    SELECT product_id,
           COUNT(*) AS num_purchase_add_to_carts
    FROM products_purchase_sessions
    -- get add-to-cart event from sessions in which product was added to cart
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* count purchases */
product_purchases AS (
    SELECT product_id,
           COUNT(DISTINCT(session_id)) AS num_purchases
    FROM products_purchase_sessions
    -- get add-to-cart events since only products in a cart can be purchased
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* count number of add to carts */
product_add_to_carts AS (
    SELECT product_id,
           num_purchases,
           num_carts
    FROM (
        SELECT pa.product_id,
               IFNULL(
                   SUM(npa.num_non_purchase_add_to_carts), 0
               ) AS num_non_purchase_carts,
               IFNULL(
                   SUM(ppa.num_purchase_add_to_carts), 0
               ) AS num_purchase_carts,
               num_non_purchase_carts+num_purchase_carts AS num_carts,
               MAX(pa.num_purchases) AS num_purchases
        FROM product_purchase_add_to_cart ppa
        LEFT JOIN product_non_purchase_add_to_cart npa USING (product_id)
        LEFT JOIN product_purchases pa USING (product_id)
        GROUP BY pa.product_id
        ORDER BY product_id
    )
)
SELECT *
FROM product_add_to_carts

#### Do Not include Number of Purchases

The number of purchases per product is excluded here since it is captured in other intermediate queries

In [ ]:
%%sql
/* get events for sessions that did not end in a purchase */
WITH products_non_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_non_purchases_filtered
),
/* get events for sessions that did end in (convert to) a purchase */
products_purchase_sessions AS (
    SELECT session_id,
           product_id,
           event_type
    FROM int_product_purchases_filtered
),
/* count number of sessions not ending in a purchase in which product page was
viewed */
product_non_purchase_add_to_cart AS (
    SELECT product_id,
           COUNT(*) AS num_non_purchase_add_to_carts
    FROM products_non_purchase_sessions
    -- get the add-to-cart event from sessions in which product was added to
    -- a cart
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* count number of sessions ending in a purchase in which product page was
viewed */
product_purchase_add_to_cart AS (
    SELECT product_id,
           COUNT(*) AS num_purchase_add_to_carts
    FROM products_purchase_sessions
    -- get add-to-cart event from sessions in which product was added to cart
    WHERE event_type = 'add_to_cart'
    GROUP BY product_id
),
/* count number of add to carts */
product_add_to_carts AS (
    SELECT product_id,
           num_carts
    FROM (
        SELECT ppa.product_id,
               IFNULL(
                   SUM(npa.num_non_purchase_add_to_carts), 0
               ) AS num_non_purchase_carts,
               IFNULL(
                   SUM(ppa.num_purchase_add_to_carts), 0
               ) AS num_purchase_carts,
               num_non_purchase_carts+num_purchase_carts AS num_carts,
        FROM product_purchase_add_to_cart ppa
        LEFT JOIN product_non_purchase_add_to_cart npa USING (product_id)
        GROUP BY ppa.product_id
        ORDER BY product_id
    )
)
SELECT *
FROM product_add_to_carts

### `products/int_products_page_viewing_time_averaged`

There are sessions in which the last event before checkout is a `page_view` and not a `add_to_cart`, as shown below

In [ ]:
%%sql
WITH t1 AS (
    SELECT session_id,
           event_id,
           created_at,
           event_type,
           product_id,
           ROW_NUMBER() OVER(
               PARTITION BY session_id ORDER BY created_at DESC
           ) AS rn
    FROM stg_postgres_events
    WHERE session_id = '7eae2419-f22b-4c8e-8a1a-1047ae07b2f0'
),
t2 AS (
    SELECT DISTINCT(session_id) AS session_id
    FROM t1
    WHERE (rn = 3 AND event_type = 'page_view')
)
SELECT *
FROM t1
INNER JOIN t2 USING (session_id)
ORDER BY created_at

**Observations**

1. Since a `page_view` event could occur before the purchase, we need to get the time spent viewing that page. This means it is necessary to keep the next event after the `page_view` event (i.e. the `checkout` event page) in order to use its timestamp to calculate the time spent on the `page_view`.

In [ ]:
%%sql
/* get time spent on page for all events in a session */
WITH times_spent_on_page_per_session AS (
    SELECT session_id,
           product_id,
           event_type,
           -- get timestamp of current event in session
           created_at,
           -- get timestamp of next event in session
           LAG(created_at, -1) OVER(
               PARTITION BY session_id ORDER BY created_at
           ) AS created_at_next_page,
           -- get time difference between successive events in session
           datediff(
               'second', created_at, created_at_next_page
           ) AS time_on_page_seconds
    FROM stg_postgres_events
),
/* get average time spent on page per product */
avg_time_on_product_page AS (
    SELECT product_id,
           ROUND(AVG(time_on_page_seconds)) AS avg_time_on_page_seconds,
           CAST(
               ROUND(STDDEV(time_on_page_seconds)) AS INTEGER
           ) AS std_time_on_page_seconds
    FROM times_spent_on_page_per_session
    -- get events where the product page is being viewed
    WHERE event_type IN ('page_view')
    GROUP BY ALL
)
SELECT *
FROM avg_time_on_product_page

## Disconnect

Close connection

In [ ]:
%sql --close connection